For this final digital-cut up assignment, I wanted to create a free-verse rhyme with a spacy-generated text. However, I ran into some issues with Pincelate (couldn't load the model despite successfully importing the module), so I decided to stick with Pronouncing for the time being.

In [302]:
import pronouncing

In [303]:
import spacy

In [304]:
from pincelate import Pincelate

In [305]:
import numpy as np

In [306]:
import nltk

In [307]:
import re

In [308]:
import random

In [309]:
nlp = spacy.load('en_core_web_md')

In [310]:
text = open("alice.txt").read()

After importing all the necessary modules, I decided to use Alice in Wonderland as my source text because adding in syllabic structure and pronounciation seemed like it would get quite confusing, and Alice has always been a good text to manipulate.

In [311]:
doc = nlp(text)

In [376]:
words = [w for w in list(doc) if w.is_alpha]

Cycle through all of the words in the doc and store them in words variable.

In [377]:
def flatten_subtree(st):
    return ''.join([w.text_with_ws for w in list(st)]).strip()

Merge the subtrees.

In [378]:
subjects = [flatten_subtree(word.subtree).replace("\n", " ")
            for word in doc if word.dep_ in ('nsubj', 'nsubjpass')]
past_tense_verbs = [word.text for word in words if word.tag_ == 'VBD' and word.lemma_ != 'be']
adjectives = [word.text for word in words if word.tag_.startswith('JJ')]
nouns = [word.text for word in words if word.tag_.startswith('NN')]
prep_phrases = [flatten_subtree(word.subtree).replace("\n", " ")
                for word in doc if word.dep_ == 'prep']

Define syntactic units of the sentence.

In [379]:
import tracery
from tracery.modifiers import base_english

In [380]:
rules = {
    "origin": [
        "#subject.capitalize# #predicate#.",
        "#subject.capitalize# #predicate#.",
        "#prepphrase.capitalize#, #subject# #predicate#."
    ],
    "predicate": [
        "#verb#",
        "#verb# #nounphrase#",
        "#verb# #prepphrase#"
    ],
    "nounphrase": [
        "the #noun#",
        "the #adj# #noun#",
        "the #noun# #prepphrase#",
        "the #noun# and the #noun#",
        "#noun.a#",
        "#adj.a# #noun#",
        "the #noun# that #predicate#"
    ],
    "subject": subjects,
    "verb": past_tense_verbs,
    "noun": nouns,
    "adj": adjectives,
    "prepphrase": prep_phrases
}
grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
grammar.flatten("#origin#")

'I bit.'

Define the grammar structure (I used the grammar structure that was defined in the Tracery tutorial, since it seemed to be an exact replica of how English grammar works).

In [381]:
text_file = open("Apoem.txt", "a")

Open the empty text file to store the output in.

In [382]:
from textwrap import fill
output = " ".join([grammar.flatten("#origin#") for i in range(100)])
poemproto = (fill(output,60))
text_file.write(poemproto)
print(poemproto)

With the tea, she had the exact pack. The jury all said an
eager DAMAGES. I gave a little OUTSIDE. It declared of him.
Of such a rule at processions, I had. Of-the-way, he went.
Alice looked a print. Of pretending to be two people, you
went. In a natural way, she minded. Him asked to herself.
She made the kid In which. She said a much tears. On it, I
said at a reasonable pace. For a minute or two, they
fancied. Plates and dishes said the profits that found. YOU
succeeded. With all her knowledge of history, it turned. The
twelve jurors went an Alice. They hit of little cartwheels.
Alice had a mouth. What marked at her. With both paragraphs
1.E.1, I began without a grin. You left in a moment. He had.
With the name 'W. RABBIT', they yawned. She cheered the
paragraphs and the half. She Said with such sudden violence
that Alice quite jumped. The March Hare fanned like a snout.
They said a Queen. She puzzled by his face. Whose cause did
a stiff soldiers. As, you tried. Some of the other bird

Generate the new text based off of the grammar structure parameters, and write it into the empty text file. I set the range to 100 to make sure that I have enough content to work with in the next step.

In [383]:
def quick_phones(w):
    pfw = pronouncing.phones_for_word(w)
    if len(pfw) == 0:
        w = re.sub("[^a-z']", "", w)
        return " "
    else:
        return pfw[0]

For the quick_phones function, since I could not import the Pincelate model, I modified it so that Pincelate wouldn't be called if the word's pronounciation is not found. Because I was working with Alice in Wonderland, I was pretty confident that there wouldn't be any sort of "new" words that would require Pincelate.

In [384]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/kevindai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [385]:
sonnet_lines = [item for item in open("Apoem.txt").read().split("\n") if len(item) > 0]

Open and read through the text file that we had stored previously of the tracery-generated text.

In [393]:
picked = random.choice(sonnet_lines)
picked_words = [w for w in nltk.word_tokenize(picked) if w[0].isalpha()]
picked_rhyme = pronouncing.rhyming_part(quick_phones(picked_words[-1]))

for line in random.sample(sonnet_lines, len(sonnet_lines)):
    if picked == line:
        continue
    words = [w for w in nltk.word_tokenize(line) if w[0].isalpha()]
    if len(words) == 0:
        continue
    if pronouncing.rhyming_part(quick_phones(words[-1])) == picked_rhyme:
        print(picked)
        print(line)
        break

Nothing and the Cat. At once, Alice chanced underneath her
work. Than what it might appear to others that what you were


This part took quite a long time to understand. From what I could gather from the tutorial, in order to create two consecutive lines that rhyme (or rhyming couplets), we must choose some line from the text, get the pronounciation of the last word in the line, and then check through every other line to find a sentence that contains a word at the end that rhymes with the initial sentence.

In [387]:
from collections import defaultdict
rhyming_part_to_idx = defaultdict(lambda: defaultdict(list))
for i, line in enumerate(sonnet_lines):
    words = [w for w in nltk.word_tokenize(line) if w[0].isalpha()]
    if len(words) > 0:
        last_few = " ".join([quick_phones(w) for w in words[-3:]])
        rhyming_part = pronouncing.rhyming_part(last_few)
        rhyming_part_to_idx[rhyming_part][words[-1]].append(i)

Since we had to check through every line to find the rhyme match, the tutorial also offered an alternative when working with larger corpuses. Although Alice in Wonderland isn't that large, I decided to try and follow this method in case I ended up working with a big corpus in the future.

In [388]:
rhyme_map = {k: v for k, v in rhyming_part_to_idx.items() if len(v) > 1}

In [398]:
for i in range(7):
    rhyming_part = random.choice(list(rhyme_map.keys()))
    a_set, b_set = random.sample(list(rhyme_map[rhyming_part].keys()), 2)
    a_idx = random.choice(rhyme_map[rhyming_part][a_set])
    b_idx = random.choice(rhyme_map[rhyming_part][b_set])
    print("\n".join([sonnet_lines[a_idx], sonnet_lines[b_idx]]))

canvas. Of short charges, she did the Alice round her head.
You EDITION. The Mock Turtle in a deep, hollow tone: said
Till now, Alice began in a very humble tone, going down on
payments and credit card donations, you spoke the half. On
ready thing. Alice, came the accordance that sat. It took
She spoke. Except the King, the Queen, and Alice, it shook
the short court. It had a sorrow. The jury all had without
the ill Alice. This work gave the same Alice. Without
the house of the March Hare. I drew of the leaves: '. That
began. I did the shore. The others cried. I led just at
permission, what turned a little Mouse. I said. Who flew.
saw a free Project. I remarked of its mouth. In a few
You gave the Gryphon at me. Without considering at all this
that did as. With one finger, I said. Alice said UNDER THIS


Here is the output that I received. A bit strange since none of the sentences make sense at all, and I also forgot to remove the license summary in the txt file so that part blended in as well. However, the end of each couplet end up rhyming, which was what I was trying to experiment with. Although not entirely a flop, I do hope to fix the issue with Pincelate on my computer in order to generate actual poems with structure. 